In [1]:
import numpy as np
import copy
import tensorflow as tf
from tensorflow import keras
import time
import itertools
import os
import matplotlib.pyplot as plt
from scipy import linalg

In [2]:
# Gives rewards after T-1 steps
class Market:
    def __init__(self, C,  theta, lam, sigma, P0 = 0, T=10, N = 1000):
        '''
        Market environment  
        C is the contract paid to the agent by the principal (a Tx1 vector for now)
        theta is the temporary price impact parameter
        lam is the permanent price impact parameter
        sigma is the standard devation of the noise at each time period
        T is the number of time periods to complete trading
        P0 is the initial price
        N is the number of paths 
        '''
        self.T = T
        self.C = C
        self.theta = theta
        self.lam = lam
        self.std = sigma
        self.N = N
        self.P0 = P0

    def reset(self, C=None):
        '''
        C: Contract, function of np.array(N,T).  C should be applied by row, i.e. C([[P0,P1],[Y0,Y1]]) = [C([P0,P1]),C([Y0,Y1])]
        '''
        if C is not None:
            self.C = C

        self.Ps = np.zeros((self.N, self.T+1)) #Will fill with prices encountered in the market at times 0,...,T
        self.Xs = np.zeros((self.N, self.T)) # Shares purchased at times 1,...,T
        self.epsilons = np.random.normal(scale = self.std, size = (self.N,self.T) ) 
        self.t = 0
        self.done = False
        self.cost = np.zeros(self.N) #Cost of purchasing all the shares
        self.Ps[:,self.t] = self.P0

        return self._obs()
    
    def step(self, X):
        '''
        X: action, a scalar
        return obs, reward, t
        '''
        self.t = self.t + 1
        if self.t == self.T-1:
            self.Xs[:,self.t-1] = X
            self.Xs[:,self.t] = 1-np.sum(self.Xs,axis=1) # Must end with exactly 1 share.
            self.Ps[:,self.t] = self.Ps[:,self.t-1] + (self.theta+self.lam)*self.Xs[:,self.t-1] - self.theta*self.Xs[:,self.t-2] + self.epsilons[:,self.t-1]
            self.cost = self.cost + self.Xs[:,self.t-1]*self.Ps[:,self.t]
            self.t = self.t + 1
            self.Ps[:,self.t] = self.Ps[:,self.t-1] + (self.theta+self.lam)*self.Xs[:,self.t-1] - self.theta*self.Xs[:,self.t-2] + self.epsilons[:,self.t-1]
            self.cost = self.cost + self.Xs[:,self.t-1]*self.Ps[:,self.t]
            self.done = True
        else:
            self.Xs[:,self.t-1] = X #Weird indexing... X[0] is the shares purchased at time 1, etc.
            self.Ps[:,self.t] = self.Ps[:,self.t-1] + (self.theta+self.lam)*self.Xs[:,self.t-1] - self.theta*self.Xs[:,self.t-2] + self.epsilons[:,self.t-1]
            self.cost = self.cost + self.Xs[:,self.t-1]*self.Ps[:,self.t]
        if self.done:
            #reward = np.dot(self.Ps[:,1:],self.C)-self.cost
            reward = self.C(self.Ps[:,1:]) - self.cost
        else:
            reward = 0
        return self._obs(), reward, self.done, self.t

    def _obs(self):
        return np.concatenate([self.Ps, self.Xs],axis=1)

In [3]:
def opt(tau, theta, lam, T, gamma = 0, sigma = 0):
  # tau is a Tx1 vector of weights
  e = np.zeros(T)
  e[-1] = 1
  # Define Ahat
  r1 = np.zeros(T)
  r1[0] = 4*theta+2*lam + gamma*sigma**2
  r1[1] = -(lam+2*theta)
  c1 = np.transpose(r1)
  Ahat = linalg.toeplitz(c1)
  Ahat[-1,-1] = 1
  Ahat[-1,-2] = 0
  # Define Ehat
  r1 = np.zeros(T)
  r1[0] = lam+theta + gamma*sigma**2
  r1[1] = -theta
  c1 = np.zeros(T)
  c1[0] = r1[0]
  c1[1:] = gamma*sigma**2
  Ehat = linalg.toeplitz(c1,r1)
  Ehat[-1] = 1
  # Define F
  r1 = np.zeros(T)
  r1[0] = 1
  c1 = np.zeros(T)
  c1[0] = 1
  c1[1] = -1
  F = linalg.toeplitz(c1,r1)
  AEhat = np.matmul(np.linalg.inv(Ahat),Ehat)
  cumulativeX = np.dot(AEhat,tau)
  xopt = np.matmul(F,cumulativeX)
  Mhat = lam*AEhat + theta*np.matmul(F,AEhat)
  Mhat = Mhat + Mhat.T
  bestreward = np.dot(tau-xopt,(lam*cumulativeX+theta*xopt))
  Mones = np.dot(np.linalg.inv(Mhat),np.ones(T))
  bestcost = 1/np.dot(np.ones(T),Mones)
  copt = Mones*bestcost
  cost = np.dot(tau,np.dot(Mhat,tau))/2
  return xopt, bestreward, -cost, copt, -bestcost/2 #Returns the agent's best strategy when given the contract tau, reward from following the best strategy, the principal's reward from offering contract tau when the agent follows the best strategy, the principal's best contract, and the principal's reward when offering the best contract and the agent responds optimally.

In [4]:
class ReplayBuffer():
    '''
        Implementation of a simple Replay Buffer for TD3.
        You are free to modify this file, or implement your own replay buffer.
        However, notice that the provided starter code assumes
        implementation of this Replay Buffer.
        An important note: we store not_done, which is the negation of the
        done signal. not_done = 1 if done = 0, and vice versa.
        We also return not_done when sampling
    '''
    def __init__(self, state_dim, action_dim, max_size=int(1e6)):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0
        self.state_dim = state_dim
        self.action_dim = action_dim

        self.state = np.zeros((max_size, state_dim))
        self.action = np.zeros((max_size, action_dim))
        self.next_state = np.zeros((max_size, state_dim))
        self.reward = np.zeros((max_size, 1))
        self.not_done = np.zeros((max_size, 1))


    def add(self, state, action, next_state, reward, done):
        '''
            adds a single transition tuple to the replay buffer.
            state: np array of size (state_dim, )
            action: np array of size (action_dim, )
            next_state: np array of size (state_dim, )
            reward: float
            done: float (1.0 if done, 0.0 otherwise)
        '''
        self.state[self.ptr] = state
        self.action[self.ptr] = action
        self.next_state[self.ptr] = next_state
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def add_batch(self, state, action, next_state, reward, done):
        '''
            adds a batch of transition tuples to the replay buffer.
            state: np array of size (batch_size, state_dim)
            action: np array of size (batch_size, action_dim)
            next_state: np array of size (batch_size, state_dim)
            reward: np array of size (batch_size, )
            done: np array of size (batch_size, )
        '''
        batch_size = state.shape[0]
        if len(reward.shape) == 1:
            reward = reward.reshape((-1, 1))
        if len(done.shape) == 1:
            done = done.reshape((-1, 1))
        if batch_size >= self.max_size:
            self.state = state[batch_size - self.max_size:]
            self.action = action[batch_size - self.max_size:]
            self.next_state = next_state[batch_size - self.max_size:]
            self.reward = reward[batch_size - self.max_size:]
            self.not_done = (1. - done)[batch_size - self.max_size:]
            self.ptr = 0
            self.size = self.max_size
        # More than vacant space. Need to evict front
        elif batch_size >= self.max_size - self.size:
            vacancy = self.max_size - self.size
            remaining = batch_size - vacancy
            self.state[self.ptr:self.ptr + vacancy] = state[:vacancy]
            self.action[self.ptr:self.ptr + vacancy] = action[:vacancy]
            self.next_state[self.ptr:self.ptr + vacancy] = next_state[:vacancy]
            self.reward[self.ptr:self.ptr + vacancy] = reward[:vacancy]
            self.not_done[self.ptr:self.ptr + vacancy] = (1. - done)[:vacancy]
            self.ptr = 0
            self.state[:remaining] = state[vacancy:]
            self.action[:remaining] = action[vacancy:]
            self.next_state[:remaining] = next_state[vacancy:]
            self.reward[:remaining] = reward[vacancy:]
            self.not_done[:remaining] = (1. - done)[vacancy:]
            self.ptr += remaining
            self.size = self.max_size
        else:
            vacancy = batch_size
            self.state[self.ptr:self.ptr + vacancy] = state[:vacancy]
            self.action[self.ptr:self.ptr + vacancy] = action[:vacancy]
            self.next_state[self.ptr:self.ptr + vacancy] = next_state[:vacancy]
            self.reward[self.ptr:self.ptr + vacancy] = reward[:vacancy]
            self.not_done[self.ptr:self.ptr + vacancy] = (1. - done)[:vacancy]
            self.ptr += vacancy
            self.size += vacancy
        assert self.ptr < self.max_size


    def sample(self, batch_size):
        '''
            Samples a batch of transitions, with specified batch_size
            return them as float32 tf tensors.
        '''
        ind = np.random.randint(0, self.size, size=batch_size)

        return (
            tf.convert_to_tensor(self.state[ind].astype("float32")),
            tf.convert_to_tensor(self.action[ind].astype("float32")),
            tf.convert_to_tensor(self.next_state[ind].astype("float32")),
            tf.convert_to_tensor(self.reward[ind].astype("float32")),
            tf.convert_to_tensor(self.not_done[ind].astype("float32"))
        )

    def shuffle(self):
        '''
            Shuffles the replay buffer, without changing its contents.
        '''
        combined = np.hstack([self.state, self.action, self.next_state, self.reward, self.not_done])
        np.random.shuffle(combined)
        self.state = combined[:, :self.state_dim]
        self.action = combined[:, self.state_dim:self.state_dim + self.action_dim]
        self.next_state = combined[:, self.state_dim + self.action_dim:-2]
        self.reward = combined[:, -2:-1]
        self.not_done = combined[:, -1:]

    def sample_all(self):
        '''
            Sample all transitions in the replay buffer
            return them as float32 tf tensors.
        '''
        return (
            tf.convert_to_tensor(self.state[:self.size].astype("float32")),
            tf.convert_to_tensor(self.action[:self.size].astype("float32")),
            tf.convert_to_tensor(self.next_state[:self.size].astype("float32")),
            tf.convert_to_tensor(self.reward[:self.size].astype("float32")),
            tf.convert_to_tensor(self.not_done[:self.size].astype("float32"))
        )

fxn = "relu"
class Actor(keras.Model):
    '''
        The actor in TD3. Architecture from authors of TD3
    '''
    def __init__(self, action_dim, max_action):
        super().__init__()

        self.l1 = keras.layers.Dense(256, activation=fxn)
        self.l2 = keras.layers.Dense(256, activation=fxn)
        self.l3 = keras.layers.Dense(action_dim)
        self.max_action = max_action


    def call(self, state):
        '''
            Returns the tanh normalized action
            Ensures that output <= self.max_action
        '''
        a = self.l1(state)
        a = self.l2(a)
        #self.l3(tanh(a))
        return self.max_action * keras.activations.tanh(self.l3(a))
        #return self.max_action * tf.clip_by_value(self.l3(a),-1,1)


class Critic(keras.Model):
    '''
        The critics in TD3. Architecture from authors of TD3
        We organize both critics within the same keras.Model
    '''
    def __init__(self):
        super().__init__()

        # Q1 architecture
        self.l1 = keras.layers.Dense(256, activation=fxn)
        self.l2 = keras.layers.Dense(256, activation=fxn)
        self.l3 = keras.layers.Dense(1)

        # Q2 architecture
        self.l4 = keras.layers.Dense(256, activation=fxn)
        self.l5 = keras.layers.Dense(256, activation=fxn)
        self.l6 = keras.layers.Dense(1)


    def call(self, state, action):
        '''
            Returns the output for both critics. Using during critic training.
        '''
        sa = tf.concat([state, action], 1)

        q1 = self.l1(sa)
        q1 = self.l2(q1)
        q1 = self.l3(q1)

        q2 = self.l4(sa)
        q2 = self.l5(q2)
        q2 = self.l6(q2)
        return q1, q2


    def Q1(self, state, action):
        '''
            Returns the output for only critic 1. Used to compute actor loss.
        '''
        sa = tf.concat([state, action], 1)

        q1 = self.l1(sa)
        q1 = self.l2(q1)
        q1 = self.l3(q1)
        return q1
    
MSE = tf.keras.losses.MeanSquaredError()
class TD3():
    '''
        The TD3 main class. Wraps around both the actor and critic, and provides
        three public methods:
        train_on_batch, which trains both the actor and critic on a batch of
        transitions
        select_action, which outputs the action by actor given a single state
        select_action_batch, which outputs the actions by actor given a batch
        of states.
    '''
    def __init__(
        self,
        state_dim,
        action_dim,
        max_action,
        discount=0.99,
        tau=0.005,
        policy_noise=0.2,
        noise_clip=0.5,
        policy_freq=2,
        lr = 3e-4
    ):
        self.state_dim = state_dim
        self.actor = Actor(action_dim, max_action)
        self.actor_target = copy.deepcopy(self.actor)
        self.actor_optimizer = keras.optimizers.Adam(learning_rate=lr)

        self.critic = Critic()
        self.critic_target = copy.deepcopy(self.critic)
        self.critic_optimizer = keras.optimizers.Adam(learning_rate=lr)

        self.max_action = max_action
        self.discount = discount
        self.tau = tau
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip
        self.policy_freq = policy_freq

        self.total_it = 0



    def select_action(self, state):
        '''
            Select action for a single state.
            state: np.array, size (state_dim, )
            output: np.array, size (action_dim, )
        '''
        state = tf.convert_to_tensor(state.reshape(1, -1))
        return self.actor(state).numpy().flatten()

    def select_action_batch(self, state):
        '''
            Select action for a batch of states.
            state: np.array, size (batch_size, state_dim)
            output: np.array, size (batch_size, action_dim)
        '''
        if not tf.is_tensor(state):
            state = tf.convert_to_tensor(state)
        return self.actor(state)#.numpy() #No need to convert to numpy


    def train_on_batch(self, state, action, next_state, reward, not_done):
        '''
            Trains both the actor and the critics on a batch of transitions.
            state: tf tensor, size (batch_size, state_dim)
            action: tf tensor, size (batch_size, action_dim)
            next_state: tf tensor, size (batch_size, state_dim)
            reward: tf tensor, size (batch_size, 1)
            not_done: tf tensor, size (batch_size, 1)
            You need to implement part of this function.
        '''
        self.total_it += 1

        # Select action according to policy and add clipped noise
        noise = tf.clip_by_value(tf.random.normal(action.shape) * self.policy_noise,
                                 -self.noise_clip, self.noise_clip)
        

        next_action = tf.clip_by_value(self.actor_target(next_state) + noise,
                                       -self.max_action, self.max_action)

        # Compute the target Q value
        outa, outb = self.critic_target(next_state,next_action)
        target_Q_value = reward + tf.minimum(outa,outb)*self.discount*not_done


        # Get current Q estimates
        est1, est2 = self.critic(state, action)

        # Compute critic loss
        with tf.GradientTape() as tape:
          est1, est2 = self.critic(state, action)
          loss = MSE(target_Q_value,est1) + MSE(target_Q_value,est2)

        # Optimize the critic
        grads = tape.gradient(loss,self.critic.trainable_variables)
        self.critic_optimizer.apply_gradients(zip(grads,self.critic.trainable_variables))

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
          with tf.GradientTape() as tapeact:
            act_loss = -tf.reduce_sum(self.critic.Q1(state,self.actor(state)))/len(not_done.numpy())
          grads = tapeact.gradient(act_loss,self.actor.trainable_variables)
          self.critic_optimizer.apply_gradients(zip(grads,self.actor.trainable_variables))

          # Compute actor losses

          # Update the frozen target models
          self.critic_target.set_weights([self.tau*new + (1-self.tau)*old for new, old in zip(self.critic.get_weights(),self.critic_target.get_weights())])
          self.actor_target.set_weights([self.tau*new + (1-self.tau)*old for new, old in zip(self.actor.get_weights(),self.actor_target.get_weights())])


    def save(self, filename):
        '''
            Saves current weight of actor and critic. You may use this function for debugging.
            Do not modify.
        '''
        self.critic.save_weights(filename + "_critic")
        self.actor.save_weights(filename + "_actor")


    def load(self, filename):
        '''
            Loads current weight of actor and critic. Notice that we initialize the targets to
            be identical to the on-policy weights.
        '''
        self.critic.load_weights(filename + "_critic")
        self.critic_target = copy.deepcopy(self.critic)
        self.actor.load_weights(filename + "_actor")
        self.actor_target = copy.deepcopy(self.actor)

'''
    This module contains the FakeEnv class, which is a wrapper class around the
    PE dynamics model to help unrolling.
'''
class FakeEnv:
    '''
        A wrapper class around your dynamics model to facilitate model unrolling.
    '''
    def __init__(self, model):
        self.model = model

    def step(self, state, action):
        '''
            state: (B, X) tensor/array, action: (B, U) tensor/array
            X is dimension of state space
            U is dimensino of action space
            B is batch size
            Given state and action , the step function queries the dynamics model,
            and returns next_x (the next states), rewards and the boolean done signal,
            as numpy arrays.
            Do not modify.
        '''
        if not tf.is_tensor(state):
            state = tf.convert_to_tensor(state)
        if not tf.is_tensor(action):
            action = tf.convert_to_tensor(action)

        samples = self.model.predict(state, action) # (B, -1)
        # (B,), (B,), (B, X)
        rewards, not_done, next_x = samples[:, 0], samples[:, 1], samples[:, 2:]
        done = (1. - not_done)

        # Convert to all numpy, done should be boolean
        return next_x, rewards, (done > 0.5)

import time
import itertools
import numpy as np
import tensorflow as tf
from tensorflow import keras

class PEModel(keras.Model):
    '''
        An individual Probabilistic Neural Network.
        Multiple Networks with identical structure form the Probabilistic Ensemble.
        Notice that each PEModel network predicts the mean and variance of
        reward, done, delta_state in order.
        Therefore, the output layer has (state_dim + 1 + 1) * 2
    '''
    def __init__(self, state_dim, action_dim):
        super().__init__()

        self.l1 = keras.layers.Dense(256, activation="relu")
        self.l2 = keras.layers.Dense(256, activation="relu")
        self.l3 = keras.layers.Dense(256, activation="relu")
        # mean and variance for reward, done, delta_state (in this order)
        self.l4 = keras.layers.Dense((state_dim + 2) * 2)
        # this step to populate trainable_weights. Without this step,
        # PE.trainable_weights will be empty.
        self.forward(np.zeros((1, state_dim + action_dim)))

    def forward(self, net_input):
        '''
            Calls the network on a batch of inputs.
            net_input should have size (batch_size, state_dim+action_dim)
        '''
        out = self.l1(net_input)
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        return out

class PE():
    '''
        The probabilistic ensemble dynamics model class.
        Contains code to initialize, train and then predict with the ensemble.
        You will implement part of this class.
    '''
    def __init__(
        self,
        state_dim,
        action_dim,
        num_networks = 7,
        num_elites = 5,
        learning_rate = 1e-3,
    ):
        self.num_networks = num_networks
        self.num_elites = num_elites
        self.networks = [PEModel(state_dim, action_dim) for i in range(num_networks)]
        self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.output_dim = state_dim + 2
        # For smoothing the log-variance output
        self.max_logvar = tf.convert_to_tensor(-3 * np.ones([1, self.state_dim + 2]), \
                                               dtype=tf.float32)
        self.min_logvar = tf.convert_to_tensor(-7 * np.ones([1, self.state_dim + 2]), \
                                               dtype=tf.float32)

        self.total_it = 0
        self._model_inds = list(range(self.num_networks)) # for choosing elite models in inference!

    def get_output(self, output, ret_logvar=False):
        """
            output: tf tensor, shape (batch_size, (state_dim+2) * 2)
            Given network outputs, returns mean and log variance tf tensors if ret_logvar = True.
            mean: shape (batch_size, state_dim + 2)
            logvar: shape (batch_size, state_dim + 2)
            Do not modify
        """
        mean = output[:, 0:self.output_dim]
        raw_v = output[:, self.output_dim:]
        # Log variance smoothing
        logvar = self.max_logvar - tf.math.softplus(self.max_logvar - raw_v)
        logvar = self.min_logvar + tf.math.softplus(logvar - self.min_logvar)
        if ret_logvar: # for training
            return mean, logvar
        return mean, tf.math.exp(logvar) # for testing

    def _train_loss_one(self, network, train_in, train_targ):
        '''
            Compute the MLE Training Loss for a given Probabilistic Neural Network.
            train_in: tf tensor, shape (batch_size, state_dim + action_dim)
            tarin_targ: tf tensor, shape (batch_size, state_dim + 2), target output
            This function should compute the Gaussian MLE loss, summed across the entire batch.
        '''
        #raise NotImplementedError
        mean, var = self.get_output(network.forward(train_in))
        train_loss = tf.reduce_sum((mean-train_targ)**2/var,0) + tf.reduce_sum(tf.math.log(var))

        # regularization step. populate train_loss with correct Gaussian MLE loss
        train_loss += 0.01 * (tf.math.reduce_sum(self.max_logvar) - \
                      tf.math.reduce_sum(self.min_logvar))
        return train_loss


    def _MSE_loss(self, valid_in, valid_targ, final=False):
        """
            Computes the MSE loss for each Probabilistic Neural Network, for validation only.
            valid_in: tf tensor, shape (batch_size, state_dim + action_dim), validation input
            valid_targ: tf tensor, shape (batch_size, state_dim + 2), validation target
            Do not modify.
        """
        mse_losses = np.zeros(self.num_networks)
        rew_losses = np.zeros(self.num_networks)
        not_done_losses = np.zeros(self.num_networks)
        dynamics_losses = np.zeros(self.num_networks)

        for i, network in enumerate(self.networks):
            mean, _ = self.get_output(network.forward(valid_in), ret_logvar=True)
            if final:
                mse_loss = tf.reduce_mean(((mean - valid_targ) ** 2), 0)
                rew_loss = mse_loss[0]
                not_done_loss = mse_loss[1]
                dynamics_loss = tf.reduce_mean(mse_loss[2:], 0)
                mse_losses[i] = tf.reduce_mean(mse_loss, 0)
                rew_losses[i] = rew_loss
                not_done_losses[i] = not_done_loss
                dynamics_losses[i] = dynamics_loss
            else:
                mse_loss = tf.reduce_mean((mean - valid_targ) ** 2, 0)
                mse_losses[i] = tf.reduce_mean(mse_loss, 0)
        if final:
            return mse_losses, rew_losses, not_done_losses, dynamics_losses
        return mse_losses

    def _prepare_dataset(self, buffer):
        '''
            Given a replay buffer containing real environment transitions,
            prepare a dataset for training the PE of neural networks.
            The dataset contains ALL transitions in the replay buffer.
            Do not modify.
            inputs: tf tensor, shape (buffer_size, state_dim + action_dim)
            targets: tf tensor, shape (buffer_size, state_dim + 2)
        '''
        state, action, next_state, reward, not_done = buffer.sample_all() # already shuffled

        delta_state = next_state - state
        inputs = tf.concat((state, action), -1)
        targets = tf.concat((reward, not_done, delta_state), -1)
        # Both TF tensors
        return inputs, targets

    def _start_train(self, max_epochs_since_update):
        '''
            Setup some internal bookkeeping variables to determine convergence.
            Do not modify.
        '''
        self._snapshots = np.array([1e10 for i in range(self.num_networks)])
        self._epochs_since_update = 0
        self._max_epochs_since_update = max_epochs_since_update

    def _end_train(self):
        '''
            Book keeping and console output. Do not modify.
        '''
        sorted_inds = np.argsort(self._snapshots)
        self._model_inds = sorted_inds[:self.num_elites].tolist() # first elite models
        print('Final holdout_losses: ', self._snapshots)
        print('Model MSE', np.mean(self._snapshots[self._model_inds]))
        print('Rew MSE', np.mean(self._reward_mse[self._model_inds]))
        print('Not Done MSE', np.mean(self._not_done_mse[self._model_inds]))
        print('Dyn MSE', np.mean(self._dynamics_mse[self._model_inds]))

    def _save_best(self, epoch, holdout_losses):
        '''
            Determines the stopping condition for PE model training.
            The training is determined to have converged if for max_epochs_since_update epochs,
            no network in the ensemble has improved for more than 1%.
            Do not modify.
        '''
        updated = False
        for i in range(len(holdout_losses)):
            current = holdout_losses[i]
            best = self._snapshots[i]
            improvement = (best - current) / best
            if improvement > 0.01: # if decrease over 1%, save
                self._snapshots[i] = current
                #self._save_model(i)
                updated = True
                # improvement = (best - current) / best
                print('epoch {} | updated {} | improvement: {:.4f} | best: {:.4f} | current: {:.4f}'.format(\
                    epoch, i, improvement, best, current))

        if updated:
            self._epochs_since_update = 0
        else:
            self._epochs_since_update += 1

        if self._epochs_since_update > self._max_epochs_since_update:
            print('[ PE ] Breaking at epoch {}: {} epochs since update ({} max)'.format(epoch,
                self._epochs_since_update, self._max_epochs_since_update))
            return True
        else:
            return False


    def train(self, buffer, batch_size=256, holdout_ratio=0.2, max_logging=5000,
                max_grad_updates=None, max_t=None, max_epochs_since_update=5):
        '''
            For model training, uses all transitions in real buffer, and train to convergence
            in valid set. You will implement part of this training function.
        '''
        self._start_train(max_epochs_since_update)
        inputs, targets = self._prepare_dataset(buffer)

        # Split into training and holdout sets
        num_holdout = min(int(inputs.shape[0] * holdout_ratio), max_logging) 
        inputs, holdout_inputs = inputs[num_holdout:], inputs[:num_holdout]
        targets, holdout_targets = targets[num_holdout:], targets[:num_holdout]

        print('[ Euler PE ] Training {} | Target {} | Holdout: {}'.format(inputs.shape, targets.shape,
                holdout_inputs.shape))

        idxs = tf.convert_to_tensor(np.random.randint(inputs.shape[0], size=(inputs.shape[0],)))
        num_batch = int(np.ceil(idxs.shape[-1] / batch_size))

        # global counter
        t0 = time.time()
        grad_updates = 0

        for epoch in itertools.count(): # infinite loop
            for batch_num in range(num_batch):
                batch_idxs = idxs[batch_num * batch_size:(batch_num + 1) * batch_size]
                # (N, <=B): will include the remainder batch even if out of bounds!
                train_in = tf.gather(inputs, batch_idxs)
                train_targ = tf.gather(targets, batch_idxs)
                
                # For each network, get loss, compute gradient of loss
                # And apply optimizer step.
                for network in self.networks:
                    with tf.GradientTape() as tape:
                        loss = self._train_loss_one(network,train_in,train_targ)
                    grads = tape.gradient(loss,network.trainable_variables)
                    self.optimizer.apply_gradients(zip(grads,network.trainable_variables))
                
                grad_updates += 1

            idxs = tf.random.shuffle(idxs) # shuffle its dataset for each model

            # validate each model using same valid set
            holdout_losses = self._MSE_loss(holdout_inputs, holdout_targets) # (N,)
            break_train = self._save_best(epoch, holdout_losses)
            print("[ PE ] holdout_losses: ", f"Epoch {epoch}", holdout_losses) # write to log.txt

            t = time.time() - t0
            if break_train or (max_grad_updates and grad_updates > max_grad_updates):
                print('Breaking for grad updates, total number = ' + str(grad_updates))
                break

            if max_t and t > max_t:
                print('Breaking because of timeout: {}! (max: {})'.format(t, max_t))
                break

        self._snapshots, self._reward_mse, self._not_done_mse, self._dynamics_mse \
            = self._MSE_loss(holdout_inputs, holdout_targets, final=True)

        self._end_train()
        print(f"End of Model training {epoch} epochs and time {t:.0f}s")
        print('Model training epoch', epoch)
        print('Model training time', int(t))
        return grad_updates

    ### Rollout / Inference Code

    def _prepare_input(self, state, action):
        '''
            Prepares inputs for inference.
            state: tf tensor, size (batch_size, state_dim) or (state_dim, )
            action: tf tensor, size (batch_size, action_dim) or (action_dim, )
            inputs: tf tensor, size (batch_size, state_dim + action_dim)
            Do not modify.
        '''
        if state.ndim == 1:
            state = tf.expand_dims(state, 0)
        if action.ndim == 1:
            action = tf.expand_dims(action, 0) \
                     if action.shape[0] == self.action_dim else tf.expand_dims(action, 1)
        inputs = tf.concat((state, action), -1)
        assert inputs.ndim == 2
        return inputs

    def _random_inds(self, batch_size):
        '''
            Uniformly randomly pick one *elite* model for each (state, action) in batch.
            This may help you implement predict.
        '''
        inds = np.random.choice(self._model_inds, size=batch_size)
        return inds

    def predict(self, state, action, deterministic=False):
        '''
            Predicts next states, rewards and not_done using the probabilistic ensemble
            For each (state, action) pair, pick a elite model uniformly at random, then
            use that elite model to predict next state, reward and not_done. The model
            can de different for each sample in the batch.
            If deterministic=True, then the prediction should simply be the predicted mean.
            If deterministic=False, then the prediction should be sampled from N(mean, var),
            where mean is the predicted mean and var is the predicted variance.
            state: tf tensor, shape (batch_size, state_dim) or (state_dim, )
            action: tf tensor, shape (batch_size, action_dim) or (action_dim, )
            samples (return value): np array, shape (batch_size, state_dim+2)
            samples[:, 0] should be the rewards, samples[:, 1] should be the not-done signals,
            and samples[:, 2:] should be the next states.
        '''
        inputs = self._prepare_input(state, action)
        batch_size = state.shape[0]
        inds = self._random_inds(batch_size)
        shaper = np.arange(batch_size).reshape((batch_size,1))
        # Maximize vectorization by doing predictions by network
        vmean, var = self.get_output(self.networks[0].forward(inputs[inds == 0]))
        means = tf.scatter_nd(shaper[inds == 0],vmean,tf.TensorShape((batch_size,self.output_dim)))
        vars = tf.scatter_nd(shaper[inds == 0],var,tf.TensorShape((batch_size,self.output_dim)))
        for i in range(1,self.num_networks):
          vmean, var = self.get_output(self.networks[i].forward(inputs[inds == i]))
          means = tf.tensor_scatter_nd_update(means,shaper[inds == i],vmean)
          vars = tf.tensor_scatter_nd_update(vars,shaper[inds == i],var)
        if deterministic:
          samples = means
        else:
          samples = tf.random.normal(shape=means.shape,mean=means,stddev = tf.sqrt(vars))
        #raise NotImplementedError
        samples = tf.concat((samples[:,:2],state + samples[:,2:]),1)
        return samples 


In [5]:
'''
    Main class for MBPO/TD3. Contains the training routine for both MBPO and TD3,
    as well as model rollout, evaluation, and graphing functions.
    You will implement part of this file.
'''
# pylint: disable=W0201, C0103,
import os
import numpy as np
import tensorflow as tf
#import gym
import matplotlib.pyplot as plt


class MBPO:
    '''
        The main class for both TD3 and MBPO. Some of the attributes are only
        used for MBPO and not for TD3. But notice that the vast majority
        of code is shared.
    '''
    def __init__(self, train_kwargs, model_kwargs, TD3_kwargs, market_kwargs):
        # shared training parameters
        self.enable_MBPO = train_kwargs["enable_MBPO"]
        self.policy_name = train_kwargs["policy"]
        self.env_name = train_kwargs["env_name"]
        self.seed = train_kwargs["seed"] #random-seed
        self.load_model = train_kwargs["load_model"]
        self.max_timesteps = train_kwargs["max_timesteps"] #maximum real-env timestemps
        self.start_timesteps = train_kwargs["start_timesteps"] #burn-in period
        self.batch_size = train_kwargs["batch_size"]
        self.eval_freq = train_kwargs["eval_freq"] #Model evaluation frequency
        self.save_model = train_kwargs["save_model"]
        self.expl_noise = train_kwargs["expl_noise"] #TD3 exploration noise
        self.max_episodes = train_kwargs["max_episodes"]
        self.OU_enabled = train_kwargs["OU_enabled"]
        self.sigma = train_kwargs["sigma"]
        self.theta = train_kwargs["theta"]
        self.dt = train_kwargs["dt"]
        self.normal_buffer_size = 1000000
        self.param_noise = train_kwargs["param_noise"]
        if self.OU_enabled:
          self.rho = self.sigma**2/(2*self.theta)*np.exp(-self.theta*self.dt)
          self.gamma = np.sqrt(1-self.rho**2)
          #self.normal_buffer_size = normal_buffer_size
          self.normal_buffer = np.random.normal(scale = self.gamma, size = self.normal_buffer_size)
          self.used_normals = 1
          self.noise = self.normal_buffer[0]

        # MBPO parameters. Pseudocode refers to MBPO pseudocode in writeup.
        self.model_rollout_batch_size = train_kwargs["model_rollout_batch_size"]
        self.num_rollouts_per_step = train_kwargs["num_rollouts_per_step"] #M in pseudocode
        self.rollout_horizon = train_kwargs["rollout_horizon"] #k in pseudocode
        self.model_update_freq = train_kwargs["model_update_freq"] #E in pseudocode
        self.num_gradient_updates = train_kwargs["num_gradient_updates"] #G in pseudocode
        self.percentage_real_transition = train_kwargs["percentage_real_transition"]
        self.verbose = train_kwargs["verbose"]
        self.paramsigma = train_kwargs["paramsigma"] # Exploration is done with parameter space noise, "paramsigma" is the variance of the noise added to each parameter
        

        # TD3 agent parameters
        self.discount = TD3_kwargs["discount"] #discount factor
        self.tau = TD3_kwargs["tau"] #target network update rate
        self.policy_noise = TD3_kwargs["policy_noise"] #sigma in Target Policy Smoothing
        self.noise_clip = TD3_kwargs["noise_clip"] #c in Target Policy Smoothing
        self.policy_freq = TD3_kwargs["policy_freq"] #d in TD3 pseudocode
        self.lr = TD3_kwargs["lr"]
        #self.use_OU = TD3_kwargs["OU_enabled"]

        # Dynamics model parameters
        self.num_networks = model_kwargs["num_networks"] #number of networks in ensemble
        self.num_elites = model_kwargs["num_elites"] #number of elites used to predict
        self.model_lr = model_kwargs["model_lr"] #learning rate for dynamics model

        # Since dynamics model remains unchanged every epoch
        # We can perform the following optimization:
        # instead of sampling M rollouts every step for E steps, sample B * M rollouts per
        # epoch, where each epoch is just E environment steps.
        self.rollout_batch_size = self.model_rollout_batch_size * self.num_rollouts_per_step
        # Number of steps in FakeEnv
        self.fake_env_steps = 0
        # Save these computations
        self.real_transitions = round(self.batch_size*self.percentage_real_transition)
        self.model_transitions = self.batch_size - self.real_transitions

        self.C = market_kwargs["Contract"]
        self.mktsigma = market_kwargs["Sigma"]
        self.mktlam = market_kwargs["Lam"]
        self.mktN = market_kwargs["N"] # Essentially how many rollouts to do with the policy between updates
        self.mktT = market_kwargs["T"]
        self.mkttheta = market_kwargs["Theta"]
        self.mktgamma = market_kwargs["Gamma"]
        self.minut = market_kwargs["minut"]
        self.mktP0 = market_kwargs["P0"]
        self.evalN = market_kwargs["evalN"] # How many rollouts to take the average over when evaluating the agent
        self.tol = market_kwargs["tol"] # Used for convergence criteria: Agent is done if they get at least tol% of the reward they would have from acting optimally

    def eval_policy(self, eval_episodes=1):
        '''
            Runs policy for eval_episodes and returns average reward.
            A fixed seed is used for the eval environment.
            Do not modify.
        '''
        env_name = self.env_name
        policy = self.policy

        eval_env = Market(self.C,self.mkttheta, self.mktlam, self.mktsigma, self.mktP0, self.mktT, self.evalN*eval_episodes)
        
        #def optresponse(s): #Given the state, returns the action of the analytic optimal action
            #N = len(s)
            #T = int((len(s[0])-1)/2)
            #t = T-(np.sum(s[0][1:T+1] == 0))
            #return self.bestresponse[t]*np.ones(N)
        

        state, done = eval_env.reset(), False
        '''
            Will compute the ratio of the average reward obtained from the learned policy and the average rewards that
            would have been obtained from acting with the analytically optimal strategy.  We compare them acting with the
            same set of epsilons to minimize the impact of the randomness in the environment when evaluating.
        '''
        # Find the rewards obtained by the analytic optimal policy
        #PI = self.mkttheta*self.bestresponse+self.mktlam*np.cumsum(self.bestresponse)
        #bestrewards = np.dot((eval_env.epsilons+PI),self.C-self.bestresponse)
        # Find the rewards from the current learned policy
        while not done:
            action = policy.select_action_batch(state)
            state, reward, done, _ = eval_env.step(np.array(action).flatten())
        eval_reward = (np.mean(self.util(reward)))#/np.mean(bestrewards) 

        print("---------------------------------------")
        print(f"Agent Evaluation over {eval_episodes} episodes: {eval_reward:.3f}")
        print("---------------------------------------")
        return eval_reward

    def util(self,x):
      return -np.exp(-self.mktgamma*np.maximum(x,self.minut))


    def init_models_and_buffer(self):
        '''
            Initialize the PE dynamics model, the TD3 policy, and the two replay buffers.
            The PE dynamics model and the replay_buffer_Model will not be used if MBPO is disabled.
            Do not modify.
        '''
        self.file_name = f"{self.policy_name}_{self.env_name}_{self.seed}"
        print("---------------------------------------")
        print(f"Policy: {self.policy_name}, Env: {self.env_name}, Seed: {self.seed}")
        print("---------------------------------------")

        if not os.path.exists("./results"):
            os.makedirs("./results")

        if self.save_model and not os.path.exists("./models"):
            os.makedirs("./models")


        env = Market(self.C, self.mkttheta, self.mktlam, self.mktsigma, self.mktP0, self.mktT, self.mktN)
        state_dim = 2*self.mktT + 1 
        action_dim = 1
        max_action = 1

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.max_action = max_action

        td3_kwargs = {
            "state_dim": state_dim,
            "action_dim": action_dim,
            "max_action": max_action,
            "discount": self.discount,
            "tau": self.tau,
            "lr": self.lr
        }

        # Target policy smoothing is scaled wrt the action scale
        td3_kwargs["policy_noise"] = self.policy_noise * max_action
        td3_kwargs["noise_clip"] = self.noise_clip * max_action
        td3_kwargs["policy_freq"] = self.policy_freq

        model_kwargs = {
            "state_dim": state_dim,
            "action_dim": action_dim,
            "num_networks": self.num_networks,
            "num_elites": self.num_elites,
            "learning_rate": self.model_lr,
        }

        self.policy = TD3(**td3_kwargs) #TD3 policy
        self.model = PE(**model_kwargs) #Dynamics model
        self.action_policy = Actor(self.action_dim, self.max_action)
        self.fake_env = FakeEnv(self.model) #FakeEnv to help model unrolling

        if self.load_model != "":
            policy_file = self.file_name if self.load_model == "default" else self.load_model
            self.policy.load(f"./models/{policy_file}")

        self.replay_buffer_Env = ReplayBuffer(state_dim, action_dim)
        self.replay_buffer_Model = ReplayBuffer(state_dim, action_dim)


    def get_action_policy(self, state):
        '''
            Adds exploration noise to an action returned by the TD3 actor.
        '''
        if self.OU_enabled:
          self.noise = self.rho*self.noise + self.normal_buffer[self.used_normals]
          self.used_normals = self.used_normals + 1
          if self.used_normals == self.normal_buffer_size:
            self.normal_buffer = np.random.normal(scale = self.gamma,size=normal_buffer_size)
            self.used_normals = 0
          action = self.policy.select_action(np.array(state)) + self.noise
        elif self.param_noise:
          action = self.action_policy(tf.convert_to_tensor(state.reshape(self.mktN, -1))).numpy().flatten()
        else:
          action = (
              self.policy.select_action(np.array(state))
              + np.random.normal(0, self.max_action * self.expl_noise, size=self.action_dim)
          ).clip(-self.max_action, self.max_action)
        return action

    def get_action_policy_batch(self, state):
        '''
            Adds exploration noise to a batch of actions returned by the TD3 actor.
        '''
        assert len(state.shape) == 2 and state.shape[1] == self.state_dim
        if self.param_noise:
          action = self.action_policy(tf.convert_to_tensor(state.reshape(self.mktN, -1)))
        else:
          action = tf.clip_by_value( 
              self.policy.select_action_batch(state) 
              + np.random.normal(0, self.max_action * self.expl_noise,
                                size=(state.shape[0], self.action_dim)),-self.max_action, self.max_action)
        # Adds the same random value to all actions in the batch
        #action = tf.clip_by_value( 
        #    self.policy.select_action_batch(state) 
        #    + np.random.normal(0, self.max_action * self.expl_noise),-self.max_action, self.max_action)
        # Numpy array!
        #You convert state to a numpy array here, and the first thing the function does is convert it back to a tensor.  No need to convert to numpy
        return np.array(action).flatten()#.astype(np.float32)

    def model_rollout(self):
        '''
            This function performs the model-rollout in batch mode for MBPO.
            This rollout is performed once per epoch, and we sample B * M rollouts.
            First, sample B * M transitions from the real environment replay buffer.
            We get B * M states from these transitions.
            Next, predict the action with exploration noise at these states using the TD3 actor.
            Then, use the step() function in FakeEnv to get the next state, reward and done signal.
            Add the new transitions from model to the model replay buffer.
            Continue until you rollout k steps for each of your B * M starting states, or you
            reached episode end for all starting states.
        '''
        rollout_batch_size = self.rollout_batch_size
        print('[ Model Rollout ] Starting  Rollout length: {} | Batch size: {}'.format(
            self.rollout_horizon, rollout_batch_size
        ))
        unit_batch_size = self.model_rollout_batch_size

        batch_pass = self.num_rollouts_per_step

        # populate this variable with total number of model transitions collected
        total_steps = rollout_batch_size
        sampled_states, _, _,_,_ =  self.replay_buffer_Env.sample(rollout_batch_size) 
        action = self.get_action_policy_batch(sampled_states)
        pred_next, pred_rewards, pred_done = self.fake_env.step(sampled_states,action) 
        self.replay_buffer_Model.add_batch(sampled_states.numpy(), action.numpy(), pred_next.numpy(), pred_rewards.numpy(), pred_done.numpy()) 

        print('[ Model Rollout ] Added: {:.1e} | Model pool: {:.1e} (max {:.1e})'.format(
            total_steps, self.replay_buffer_Model.size, self.replay_buffer_Model.max_size
        ))

        self.fake_env_steps += total_steps

    def prepare_mixed_batch(self):
        '''
            TODO: implement the mixed batch for MBPO
            Prepare a mixed batch of state, action, next_state, reward and not_done for TD3.
            This function should output 5 tf tensors:
            state, shape (self.batch_size, state_dim)
            action, shape (self.batch_size, action_dim)
            next_state, shape (self.batch_size, state_dim)
            reward, shape (self.batch_size, 1)
            not_done, shape (self.batch_size, 1)
            If MBPO is enabled, each of the 5 tensors should a mixture of samples from the
            real environment replay buffer and model replay buffer. Percentage of samples
            from real environment should match self.percentage_real_transition
            If MBPO is disabled, then simply sample a batch from real environment replay buffer.
        '''
        if self.enable_MBPO:
          rstate, raction, rnext_state, rreward, rnot_done = self.replay_buffer_Env.sample(self.real_transitions)
          mstate, maction, mnext_state, mreward, mnot_done = self.replay_buffer_Model.sample(self.model_transitions)
          state = tf.concat((rstate,mstate),0)
          action = tf.concat((raction,maction),0)
          next_state = tf.concat((rnext_state,mnext_state),0)
          reward = tf.concat((rreward,mreward),0)
          not_done = tf.concat((rnot_done,mnot_done),0)
        else:
          state, action, next_state, reward, not_done = self.replay_buffer_Env.sample(self.batch_size)
        return state, action, next_state, reward, not_done

    def plot_training_curves(self, evaluations, evaluate_episodes, evaluate_timesteps):
        '''
            Plotting script. You should include these plots in the writeup.
            Do not modify.
        '''
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        ax1.plot(evaluate_episodes, evaluations)
        ax1.set_xlabel("Training Episodes")
        ax1.set_ylabel("Evaluation Reward")
        ax1.set_title("Reward vs Training Episodes")
        ax2.plot(evaluate_timesteps, evaluations)
        ax2.set_xlabel("Training Timesteps")
        ax2.set_ylabel("Evaluation Reward")
        ax2.set_title("Reward vs Training Timesteps")
        if self.enable_MBPO:
            algo_str = "MBPO"
        else:
            algo_str = "TD3"
        fig.suptitle("Training Curves for " + algo_str, fontsize=20)
        #fig.savefig("./results/training_curve_{}.png".format(algo_str))

    def train(self, contract):
        '''
            Main training loop for both TD3 and MBPO. See Figure 2 in writeup.
        '''
        self.C = contract
        env = Market(self.C,self.mkttheta, self.mktlam, self.mktsigma, self.mktP0, self.mktT, self.mktN)
        #res = opt(self.C,self.mkttheta,self.mktlam,self.mktT)
        #self.bestresponse = res[0]
        #print("     Training agent with contract", self.C)
        print("     Beginning to train agent with new contract")
        # The replay buffers are cleared out because the rewards they contained are no longer accurate for the new contract
        self.replay_buffer_Env = ReplayBuffer(self.state_dim, self.action_dim) 
        self.replay_buffer_Model = ReplayBuffer(self.state_dim, self.action_dim) 
        

        # Evaluate untrained policy
        evaluations = [self.eval_policy()]
        self.start_episodes = self.start_timesteps/self.mktN

        evaluate_timesteps = [0]
        evaluate_episodes = [0]

        state, done = env.reset(), False
        if self.param_noise:
          self.action_policy(tf.convert_to_tensor(state.reshape(self.mktN, -1))) 

        episode_reward = 0
        episode_timesteps = 0
        episode_num = 0
        train_episodes = 0
        

        if self.enable_MBPO:
          self.start_timesteps = max(self.start_timesteps, self.model_update_freq)
        else:
          self.num_gradient_updates = 1

        for t in range(int(self.max_timesteps)):

            episode_timesteps += 1

            # Select action randomly or according to policy
            if t < self.start_timesteps:
                action = np.random.rand(self.mktN)*2-1
            else:
                action = self.get_action_policy_batch(state)
                if self.enable_MBPO and (t % self.model_update_freq == 0):
                  self.model.train(self.replay_buffer_Env)
                  self.model_rollout()

            # Perform action
            new_state, reward, done, info = env.step(action)
            reward = self.util(reward)
            #reward = reward/np.mean(-reward)

            # Store data in replay buffer
            self.replay_buffer_Env.add_batch(state, action.reshape(self.mktN,1), new_state, reward*np.ones(self.mktN), float(done)*np.ones(self.mktN))
            state = new_state

            # Train agent after collecting sufficient data
            # Perform multiple gradient steps per environment step for MBPO
            if t >= self.start_timesteps:
              for gstep in range(self.num_gradient_updates):
                states, actions, next_states, rewards, not_dones = self.prepare_mixed_batch()
                self.policy.train_on_batch(states, actions, next_states, rewards, not_dones)


            episode_reward = episode_reward + np.mean(reward)

            if done:
                # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
                if self.verbose:
                    print(f"     Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
                # Reset environment
                state, done = env.reset(), False
                if self.param_noise:
                    self.action_policy.set_weights([wt + np.random.normal(scale = self.paramsigma, size = wt.shape) for wt in self.policy.actor.get_weights()])
                episode_reward = 0
                episode_timesteps = 0
                episode_num += 1
                if t >= self.start_timesteps:
                  train_episodes = train_episodes + 1

            # Evaluate episode
            if (t + 1) % self.eval_freq == 0:
                evaluations.append(self.eval_policy())
                evaluate_episodes.append(episode_num+1)
                evaluate_timesteps.append(t+1)
                if (len(evaluations) > 3+self.start_episodes and np.std(evaluations[-5:]) < 0.005): #or (len(evaluations) > 1 and evaluations[-1] > 0.95):
                    # Can uncomment this line if you are training just the agent.
                    # If you are training the principal, plotting the training curves for the agent will eat all of your computer's memory because they don't get cleared out when the agent is reset.
                    #self.plot_training_curves(evaluations, evaluate_episodes, evaluate_timesteps)
                    print("     Agent converged successfully")
                    return evaluations, evaluate_episodes, evaluate_timesteps
                np.save(f"./results/{self.file_name}", evaluations)
                if self.save_model:
                    self.policy.save(f"./models/{self.file_name}")

            if train_episodes >= self.max_episodes:
              evaluations.append(self.eval_policy())
              evaluate_episodes.append(episode_num+1)
              evaluate_timesteps.append(t+1)
              #self.plot_training_curves(evaluations, evaluate_episodes, evaluate_timesteps)
              return evaluations, evaluate_episodes, evaluate_timesteps

In [6]:
class PrincGym:
    def __init__(self, agent_policy, theta, lam, sigma, P0 = 0, T=5, N = 10000):
        '''
        Principal's environment.  Fairly bairbones to the point I'm not sure it's really needed.
        agent_policy is the policy that the agent follows, i.e. a function that maps states to scalars
        theta is the temporary price impact parameter
        lam is the permanent price impact parameter
        sigma is the standard devation of the noise at each time period
        T is the number of time periods to complete trading
        P0 is the initial price
        N is the number of paths 
        '''
        self.T = T
        self.theta = theta
        self.lam = lam
        self.std = sigma
        self.N = N
        self.P0 = P0
        self.policy = agent_policy
        self.sigma = sigma
        self.Ps = np.array([1])
        self.t = 0
        

    def reset(self, C=None):
        '''
        C: Contract, function of np.array(N,T).  C should be applied by row, i.e. C([[P0,P1],[Y0,Y1]]) = [C([P0,P1]),C([Y0,Y1])]
        '''
        if C is not None:
            self.C = C

        self.Ps = np.array([1]) #Will fill with prices encountered in the market at times 0,...,T
        self.t = 0
        self.done = False

        return self._obs()
    
    def step(self, C):
        '''
        C: Principal's Action, a T dimensional vector
        return obs, reward, done, t
        Steps through the market environment where the agent is acting according to agent_policy
        '''
        tau = C
        env = Market(tau, self.theta, self.lam, self.sigma, self.P0, self.T, self.N)
        s, done = env.reset(), False
        while not done:
          act = self.policy(s)
          s, _, done, _ = env.step(act)
        Prices = s[:,1:self.T+1]
        self.Ps = Prices
        self.done = np.ones(self.N)
        reward = -tau(Prices) # The negative of the amount the principal pays the agent
        self.t = 0
        return self._obs(), reward, self.done, self.t

    def _obs(self):
        return self.Ps

In [7]:
'''
    Main class for MBPO/TD3. Contains the training routine for both MBPO and TD3,
    as well as model rollout, evaluation, and graphing functions.
    You will implement part of this file.
'''


class MBPOP:
    '''
        The main class for both TD3 and MBPO. Some of the attributes are only
        used for MBPO and not for TD3. But notice that the vast majority
        of code is shared.
    '''
    def __init__(self, train_kwargs, model_kwargs, TD3_kwargs, market_kwargs, agent_v):
        # shared training parameters
        self.enable_MBPO = train_kwargs["enable_MBPO"]
        self.policy_name = train_kwargs["policy"]
        self.env_name = train_kwargs["env_name"]
        self.seed = train_kwargs["seed"] #random-seed
        self.load_model = train_kwargs["load_model"]
        self.max_timesteps = train_kwargs["max_timesteps"] #maximum real-env timestemps
        self.start_timesteps = train_kwargs["start_timesteps"] #burn-in period
        self.batch_size = train_kwargs["batch_size"]
        self.eval_freq = train_kwargs["eval_freq"] #Model evaluation frequency
        self.save_model = train_kwargs["save_model"]
        self.expl_noise = train_kwargs["expl_noise"] #TD3 exploration noise
        self.max_episodes = train_kwargs["max_episodes"]
        self.OU_enabled = train_kwargs["OU_enabled"]
        self.sigma = train_kwargs["sigma"]
        self.theta = train_kwargs["theta"]
        self.dt = train_kwargs["dt"]
        self.normal_buffer_size = 1000000
        self.param_noise = train_kwargs["param_noise"]
        if self.OU_enabled:
          self.rho = self.sigma**2/(2*self.theta)*np.exp(-self.theta*self.dt)
          self.gamma = np.sqrt(1-self.rho**2)
          #self.normal_buffer_size = normal_buffer_size
          self.normal_buffer = np.random.normal(scale = self.gamma, size = self.normal_buffer_size)
          self.used_normals = 1
          self.noise = self.normal_buffer[0]
        self.tol = train_kwargs["tol"]
        self.paramsigma = train_kwargs["paramsigma"]

        # MBPO parameters. Pseudocode refers to MBPO pseudocode in writeup.
        self.model_rollout_batch_size = train_kwargs["model_rollout_batch_size"]
        self.num_rollouts_per_step = train_kwargs["num_rollouts_per_step"] #M in pseudocode
        self.rollout_horizon = train_kwargs["rollout_horizon"] #k in pseudocode
        self.model_update_freq = train_kwargs["model_update_freq"] #E in pseudocode
        self.num_gradient_updates = train_kwargs["num_gradient_updates"] #G in pseudocode
        self.percentage_real_transition = train_kwargs["percentage_real_transition"]
        

        # TD3 agent parameters
        self.discount = TD3_kwargs["discount"] #discount factor
        self.tau = TD3_kwargs["tau"] #target network update rate
        self.policy_noise = TD3_kwargs["policy_noise"] #sigma in Target Policy Smoothing
        self.noise_clip = TD3_kwargs["noise_clip"] #c in Target Policy Smoothing
        self.policy_freq = TD3_kwargs["policy_freq"] #d in TD3 pseudocode
        self.lr = TD3_kwargs["lr"]
        #self.use_OU = TD3_kwargs["OU_enabled"]

        # Dynamics model parameters
        self.num_networks = model_kwargs["num_networks"] #number of networks in ensemble
        self.num_elites = model_kwargs["num_elites"] #number of elites used to predict
        self.model_lr = model_kwargs["model_lr"] #learning rate for dynamics model

        # Since dynamics model remains unchanged every epoch
        # We can perform the following optimization:
        # instead of sampling M rollouts every step for E steps, sample B * M rollouts per
        # epoch, where each epoch is just E environment steps.
        self.rollout_batch_size = self.model_rollout_batch_size * self.num_rollouts_per_step
        # Number of steps in FakeEnv
        self.fake_env_steps = 0
        # Save these computations
        self.real_transitions = round(self.batch_size*self.percentage_real_transition)
        self.model_transitions = self.batch_size - self.real_transitions
        
        self.mktsigma = market_kwargs["Sigma"]
        self.mktlam = market_kwargs["Lam"]
        self.mktN = market_kwargs["N"]
        self.mktT = market_kwargs["T"]
        self.mkttheta = market_kwargs["Theta"]
        self.mktP0 = market_kwargs["P0"]
        self.weightavg = market_kwargs["weightavg"] # Boolean: whether the contract weights are required to sum to 1
        self.perfectagent = market_kwargs["perfectagent"] # Boolean: If true, uses the agent's analytically optimal response during training.
        self.evalN = market_kwargs["evalN"]
        self.mktgamma = market_kwargs["Gamma"]

        self.agent = MBPO(agent_v["train_kwargs"], agent_v["model_kwargs"], agent_v["TD3_kwargs"], agent_v["market_kwargs"])
        self.agent.init_models_and_buffer()


    def eval_policy(self, eval_episodes=1):
        '''
            Runs policy for eval_episodes and returns average reward.
            A fixed seed is used for the eval environment.
            Do not modify.
        '''
        env_name = self.env_name
        policy = self.policy
        #state = np.array([1])
        done = False
        #action = policy.select_action(state.reshape(1,-1))
        #if self.weightavg:
        #  contract = np.zeros(self.mktT)
        #  contract[:-1] = np.array(action)
        #  contract[-1] = 1-np.sum(contract)
        #  action = contract
        #else:
        #  contract = np.array(action)
        #action = contract
        #xopt = opt(action, self.mkttheta, self.mktlam, self.mktT)[0] #For evaluation, assume the agent acts according to the analytically optimal strategy
        #def optresponse(s):
        #  N = len(s)
        #  t = self.mktT-(np.sum(s[0][1:self.mktT+1] == 0))
        #  return xopt[t]*np.ones(N)
        
        def contract(P):
            return np.dot(P,self.best)+np.array(self.policy.select_action_batch(P)).flatten()
        
        #self.agent.C = contract
        self.agent.train(contract)
        def optresponse(s):
            return np.array(self.agent.policy.select_action_batch(s)).flatten()

        
        agent_excess = self.agent.eval_policy(eval_episodes=1)
        agent_excess = -np.log(-agent_excess)/self.mktgamma
        

        eval_env = PrincGym(optresponse, self.mkttheta, self.mktlam, self.mktsigma, self.mktP0, self.mktT, self.evalN*eval_episodes)
        

        eval_reward = np.mean(eval_env.step(contract)[1]) + agent_excess
        

        print("---------------------------------------")
        print(f"Principal Evaluation over {eval_episodes} episodes: {eval_reward:.3f}")
        print("---------------------------------------")
        return eval_reward

    def init_models_and_buffer(self):
        '''
            Initialize the PE dynamics model, the TD3 policy, and the two replay buffers.
            The PE dynamics model and the replay_buffer_Model will not be used if MBPO is disabled.
            Do not modify.
        '''
        self.file_name = f"{self.policy_name}_{self.env_name}_{self.seed}"
        print("---------------------------------------")
        print(f"Policy: {self.policy_name}, Env: {self.env_name}, Seed: {self.seed}")
        print("---------------------------------------")

        if not os.path.exists("./results"):
            os.makedirs("./results")

        if self.save_model and not os.path.exists("./models"):
            os.makedirs("./models")

        state_dim = self.mktT # The Principal's state is nothing: they have to provide a contract without knowing any of the market prices.
        #action_dim = self.mktT - 1
        action_dim = 1
        #if self.weightavg:
        #  action_dim = self.mktT - 1 # If the weights are required to sum to 1, the Principal chooses the first T-1 weights, and the last one is fixed
        #else:
        #  action_dim = self.mktT
        max_action = 1 # Require all weights are in [-1,1]

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.max_action = max_action

        td3_kwargs = {
            "state_dim": state_dim,
            "action_dim": action_dim,
            "max_action": max_action,
            "discount": self.discount,
            "tau": self.tau,
            "lr": self.lr
        }

        # Target policy smoothing is scaled wrt the action scale
        td3_kwargs["policy_noise"] = self.policy_noise * max_action
        td3_kwargs["noise_clip"] = self.noise_clip * max_action
        td3_kwargs["policy_freq"] = self.policy_freq

        model_kwargs = {
            "state_dim": state_dim,
            "action_dim": action_dim,
            "num_networks": self.num_networks,
            "num_elites": self.num_elites,
            "learning_rate": self.model_lr,
        }

        self.policy = TD3(**td3_kwargs) #TD3 policy
        self.model = PE(**model_kwargs) #Dynamics model
        self.action_policy = Actor(self.action_dim, self.max_action)
        self.fake_env = FakeEnv(self.model) #FakeEnv to help model unrolling

        if self.load_model != "":
            policy_file = self.file_name if self.load_model == "default" else self.load_model
            self.policy.load(f"./models/{policy_file}")

        self.replay_buffer_Env = ReplayBuffer(state_dim, action_dim)
        self.replay_buffer_Model = ReplayBuffer(state_dim, action_dim)


    def get_action_policy(self, state):
        '''
            Adds exploration noise to an action returned by the TD3 actor.
        '''
        if self.OU_enabled:
          self.noise = self.rho*self.noise + self.normal_buffer[self.used_normals]
          self.used_normals = self.used_normals + 1
          if self.used_normals == self.normal_buffer_size:
            self.normal_buffer = np.random.normal(scale = self.gamma,size=normal_buffer_size)
            self.used_normals = 0
          action = self.policy.select_action(np.array(state)) + self.noise
        elif self.param_noise:
          action = self.action_policy(tf.convert_to_tensor(state)).numpy().flatten()
        else:
          action = (
              self.policy.select_action(np.array(state))
              + np.random.normal(0, self.max_action * self.expl_noise, size=self.action_dim)
          ).clip(-self.max_action, self.max_action)
        return action

    def get_action_policy_batch(self, state):
        '''
            Adds exploration noise to a batch of actions returned by the TD3 actor.
        '''
        assert len(state.shape) == 2 and state.shape[1] == self.state_dim
        if self.param_noise:
          #action = self.action_policy(tf.convert_to_tensor(state.reshape(self.mktN, -1)))
          action = self.action_policy(state)
        else:
          action = tf.clip_by_value( 
              self.policy.select_action_batch(state) 
              + np.random.normal(0, self.max_action * self.expl_noise,
                                size=(state.shape[0], self.action_dim)),-self.max_action, self.max_action)
        return np.array(action).flatten()#.astype(np.float32)

    def model_rollout(self):
        '''
            This function performs the model-rollout in batch mode for MBPO.
            This rollout is performed once per epoch, and we sample B * M rollouts.
            First, sample B * M transitions from the real environment replay buffer.
            We get B * M states from these transitions.
            Next, predict the action with exploration noise at these states using the TD3 actor.
            Then, use the step() function in FakeEnv to get the next state, reward and done signal.
            Add the new transitions from model to the model replay buffer.
            Continue until you rollout k steps for each of your B * M starting states, or you
            reached episode end for all starting states.
        '''
        rollout_batch_size = self.rollout_batch_size
        print('[ Model Rollout ] Starting  Rollout length: {} | Batch size: {}'.format(
            self.rollout_horizon, rollout_batch_size
        ))
        unit_batch_size = self.model_rollout_batch_size

        batch_pass = self.num_rollouts_per_step

        # populate this variable with total number of model transitions collected
        total_steps = rollout_batch_size
        
        sampled_states, _, _,_,_ =  self.replay_buffer_Env.sample(rollout_batch_size)
        action = self.get_action_policy_batch(sampled_states)
        pred_next, pred_rewards, pred_done = self.fake_env.step(sampled_states,action) 
        self.replay_buffer_Model.add_batch(sampled_states.numpy(), action.numpy(), pred_next.numpy(), pred_rewards.numpy(), pred_done.numpy()) 

        print('[ Model Rollout ] Added: {:.1e} | Model pool: {:.1e} (max {:.1e})'.format(
            total_steps, self.replay_buffer_Model.size, self.replay_buffer_Model.max_size
        ))

        self.fake_env_steps += total_steps

    def prepare_mixed_batch(self):
        '''
            TODO: implement the mixed batch for MBPO
            Prepare a mixed batch of state, action, next_state, reward and not_done for TD3.
            This function should output 5 tf tensors:
            state, shape (self.batch_size, state_dim)
            action, shape (self.batch_size, action_dim)
            next_state, shape (self.batch_size, state_dim)
            reward, shape (self.batch_size, 1)
            not_done, shape (self.batch_size, 1)
            If MBPO is enabled, each of the 5 tensors should a mixture of samples from the
            real environment replay buffer and model replay buffer. Percentage of samples
            from real environment should match self.percentage_real_transition
            If MBPO is disabled, then simply sample a batch from real environment replay buffer.
        '''
        if self.enable_MBPO:
          rstate, raction, rnext_state, rreward, rnot_done = self.replay_buffer_Env.sample(self.real_transitions)
          mstate, maction, mnext_state, mreward, mnot_done = self.replay_buffer_Model.sample(self.model_transitions)
          state = tf.concat((rstate,mstate),0)
          action = tf.concat((raction,maction),0)
          next_state = tf.concat((rnext_state,mnext_state),0)
          reward = tf.concat((rreward,mreward),0)
          not_done = tf.concat((rnot_done,mnot_done),0)
        else:
          state, action, next_state, reward, not_done = self.replay_buffer_Env.sample(self.batch_size)
        return state, action, next_state, reward, not_done

    def plot_training_curves(self, evaluations, evaluate_episodes, evaluate_timesteps):
        '''
            Plotting script. You should include these plots in the writeup.
            Do not modify.
        '''
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        ax1.plot(evaluate_episodes, evaluations)
        ax1.set_xlabel("Training Episodes")
        ax1.set_ylabel("Evaluation Reward")
        ax1.set_title("Reward vs Training Episodes")
        ax2.plot(evaluate_timesteps, evaluations)
        ax2.set_xlabel("Training Timesteps")
        ax2.set_ylabel("Evaluation Reward")
        ax2.set_title("Reward vs Training Timesteps")
        if self.enable_MBPO:
            algo_str = "MBPO"
        else:
            algo_str = "TD3"
        fig.suptitle("Training Curves for " + algo_str, fontsize=20)
        fig.savefig("./results/training_curve_{}.png".format(algo_str))

    def train(self):
        '''
            Main training loop for both TD3 and MBPO. See Figure 2 in writeup.
        '''
        #states = np.random.rand((self.mktN,self.mktT))*2-1
        agent = self.agent
        self.best = opt(np.ones(self.mktT)/self.mktT,self.mkttheta,self.mktlam,self.mktT,self.mktgamma,self.mktsigma)[3] #The best average rewards available to the Principal; used as convergence criterion
        def contract(P):
            return np.dot(P,self.best)
        #agent.train(contract)
        xopt, agent_excess, _, _, _ = opt(self.best, self.mkttheta, self.mktlam, self.mktT,self.mktgamma,self.mktsigma) #For evaluation, assume the agent acts according to the analytically optimal strategy
        def optresponse(s):
            N = len(s)
            t = self.mktT-(np.sum(s[0][1:self.mktT+1] == 0))
            return xopt[t]*np.ones(N)
        #epsilons = np.random.normal(scale = self.mktsigma, size = (self.evalN*10,self.mktT) ) 
        PI = xopt + np.cumsum(xopt)
        agent_excess = np.dot(PI,self.best-xopt)
        #agent_excess = np.dot((eval_env.epsilons+PI),self.best-bestresponse))
        
        env = PrincGym(optresponse, self.mkttheta, self.mktlam, self.mktsigma, self.mktP0, self.mktT, self.mktN)
        state, reward, done, info = env.step(contract)
        action = np.zeros(self.mktN)
        new_state, reward, done, info = env.step(contract)
        reward = reward + agent_excess
        
        action = action.reshape((self.mktN,1))
        self.replay_buffer_Env.add_batch(state, action, new_state, reward, done)
        state = new_state
        

        # Evaluate untrained policy
        evaluations = [self.eval_policy()] #This is needed to get the actor policy set up

        evaluate_timesteps = [0]
        evaluate_episodes = [0]

        if self.param_noise:
          self.action_policy(tf.convert_to_tensor(state.reshape(self.mktN,-1)))

        episode_reward = 0
        episode_timesteps = 0
        episode_num = 0
        train_episodes = 0
        

        if self.enable_MBPO:
          self.start_timesteps = max(self.start_timesteps, self.model_update_freq)
        else:
          self.num_gradient_updates = 1

        for t in range(int(self.max_timesteps)):

            episode_timesteps += 1

            # Select action randomly or according to policy
            if t < self.start_timesteps-1:
                action = np.random.rand(self.mktN)
                #if self.weightavg:
                #  action = np.zeros(self.mktT)
                #  action[:-1] = np.random.rand(self.mktT-1)*2-1
                #  action[-1] = 1-np.sum(action)
                #else:
                #  action = np.random.rand(self.mktT)*2-1
            else:
                agent.start_timesteps = (self.mktT-1)*20 
                action = self.get_action_policy_batch(state)
                #The agent acts randomly for a few episodes instead of many now.  This is because the Principal isn't offering random contracts anymore, so the new contract they offer will be close to the old one, and (hopefully) the optimal response to the new one will be similar to the optimal response to the old one
                #action = self.get_action_policy(state.reshape(1,-1))
                #if self.weightavg:
                #  contract = np.zeros(self.mktT)
                #  contract[:-1] = np.array(action)
                #  contract[-1] = 1-np.sum(contract)
                #  action = contract    
                #if self.enable_MBPO and (t % self.model_update_freq == 0):
                #  self.model.train(self.replay_buffer_Env)
                #  self.model_rollout()
                # Perform model rollout and model training at appropriate timesteps 

            # Perform action
            if self.perfectagent:
              xopt = opt(np.array(action), self.mkttheta, self.mktlam, self.mktT)[0]
              def optresponse(s):
                N = len(s)
                #T = int((len(s[0])-1)/2)
                t = self.mktT-(np.sum(s[0][1:self.mktT+1] == 0))
                return xopt[t]*np.ones(N)
            else:
              #agent.C = np.array(action)
                def contract(P):
                    return np.array(self.get_action_policy_batch(P)).flatten() + np.dot(P,self.best)
                agent.train(contract)
                def optresponse(s):
                    return np.array(agent.policy.select_action_batch(s)).flatten()
                agent_excess = agent.eval_policy(eval_episodes=1)
                agent_excess = -np.log(-agent_excess)/self.mktgamma

            env = PrincGym(optresponse, self.mkttheta, self.mktlam, self.mktsigma, self.mktP0, self.mktT, self.mktN)
            new_state, reward, done, info = env.step(contract)
            reward = reward + agent_excess

            # Store data in replay buffer
            #if self.weightavg:
              #action = action[:-1]
            action = action.reshape((self.mktN,1))
            self.replay_buffer_Env.add_batch(state, action, new_state, reward, done)
            state = new_state

            # Train agent after collecting sufficient data
            '''if t >= self.start_timesteps:
              state, action, next_state, reward, not_done = self.prepare_mixed_batch()
              self.policy.train_on_batch(state, action, next_state, reward, not_done)'''
            # Perform multiple gradient steps per environment step for MBPO
            if t >= self.start_timesteps:
              for gstep in range(self.num_gradient_updates):
                states, actions, next_states, rewards, not_dones = self.prepare_mixed_batch()
                self.policy.train_on_batch(states, actions, next_states, rewards, not_dones)


            #raise NotImplementedError
            episode_reward = episode_reward + np.mean(reward)

            if done[0]:
                # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
                print(f"Principal Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
                # Reset environment
                done = False
                if self.param_noise:
                    self.action_policy.set_weights([wt + np.random.normal(scale = self.paramsigma, size = wt.shape) for wt in self.policy.actor.get_weights()])
                episode_reward = 0
                episode_timesteps = 0
                episode_num += 1
                if t >= self.start_timesteps:
                  train_episodes = train_episodes + 1

            # Evaluate episode
            if (t + 1) % self.eval_freq == 0:
                evaluations.append(self.eval_policy())
                evaluate_episodes.append(episode_num+1)
                evaluate_timesteps.append(t+1)
                if len(evaluations) > 5 and np.std(evaluations[-5:]) < 0.0001:
                    self.plot_training_curves(evaluations, evaluate_episodes, evaluate_timesteps)
                    print("Principal converged successfully")
                    return evaluations, evaluate_episodes, evaluate_timesteps
                np.save(f"./results/{self.file_name}", evaluations)
                if self.save_model:
                    self.policy.save(f"./models/{self.file_name}")

            if train_episodes >= self.max_episodes:
              evaluations.append(self.eval_policy())
              evaluate_episodes.append(episode_num+1)
              evaluate_timesteps.append(t+1)
              self.plot_training_curves(evaluations, evaluate_episodes, evaluate_timesteps)
              return evaluations, evaluate_episodes, evaluate_timesteps

In [8]:
T = 4
C = np.ones(T)/T
v = {}
v["train_kwargs"] = {
    "enable_MBPO": False,
    "policy": "TD3",
    "env_name": "Agent",
    "seed": 0,
    "load_model": '',
    "max_timesteps": 50000,
    "start_timesteps": 2500,
    "batch_size": 256,
    "eval_freq": 2500,
    "save_model": False,
    "expl_noise": 0.1,
    "model_rollout_batch_size": 250,
    "num_rollouts_per_step": 400,
    "rollout_horizon": 1,
    "model_update_freq": 5000,
    "num_gradient_updates": 20,
    "percentage_real_transition": 0.2,
    "max_episodes": 2000,
    "OU_enabled": False,
    "param_noise": True,
    "paramsigma": 0.01,
    "sigma": 0.2,
    "theta": 0.15,
    "dt": 0.01,
    "verbose": True}
v["market_kwargs"] = {
    "Contract": C,
    "Sigma": 0.05, # Although possibly unrealistic, having sigma small is helpful for convergence
    "Lam": 1,
    "N": 2,
    "T": T,
    "Theta": 1,
    "P0": 0,
    "Gamma": 1,
    "tol": 0.9,
    "minut": -5}
  
v["TD3_kwargs"]={
  "discount": 0.99,
  "tau": 0.005,
  "policy_noise": 0.05,
  "noise_clip": 0.5,
  "policy_freq": 2,
  "lr": 3e-5}

v["model_kwargs"]={
  "num_networks": 7,
  "num_elites": 5,
  "model_lr": 0.0001}

T = 4
tau = np.ones(T)/T
# Modifying the parameters... these seem to work fairly okay for me?
v["train_kwargs"]['paramsigma'] = 0.1
v["TD3_kwargs"]["policy_noise"] = 0.2
v["market_kwargs"]["Contract"] = tau
v["market_kwargs"]["Sigma"] = 0.05
v["market_kwargs"]["Theta"] = 1
v["market_kwargs"]["Lam"] = 1
v["market_kwargs"]["N"] = 10
v["market_kwargs"]["T"] = T
v["market_kwargs"]["MktP0"] = 0
v["market_kwargs"]["tol"] = 0.9
v["market_kwargs"]["evalN"] = 100
v["train_kwargs"]["start_timesteps"] = 500*(T-1) # 1000 episodes
v["train_kwargs"]["max_timesteps"] = 1000000
v["train_kwargs"]["max_episodes"] = 5000
v["train_kwargs"]["eval_freq"] = 100*(T-1) # Evaluate every 100 episodes
v["train_kwargs"]["verbose"] = True

In [9]:
T = 4
Pv = {}
Pv["train_kwargs"] = {
  "enable_MBPO": False,
  "policy": "TD3",
  "env_name": "Principal",
  "seed": 0,
  "load_model": '',
  "max_timesteps": 50000,#1000000,
  "start_timesteps": 3, #This is very small, but it takes a long time for the agent to converge for them.
  "batch_size": 128,
  "eval_freq": 20,
  "save_model": False,
  "expl_noise": 0.1,
  "model_rollout_batch_size": 250,
  "num_rollouts_per_step": 400,
  "rollout_horizon": 1,
  "model_update_freq": 5000,
  "num_gradient_updates": 20,
  "percentage_real_transition": 0.2,
  "max_episodes": 3000,
  "OU_enabled": False,
  "param_noise": True,
  "sigma": 0.2,
  "theta": 0.15,
  "paramsigma": 0.02, #Normally 0.01
  "tol": 1.1,
  "dt": 0.01}

Pv["market_kwargs"] = {
  "Sigma": 0.05, #Possibly unrealistic, but helps with convergence
  "Lam": 1,
  "N": 200,
  "T": T,
  "Theta": 1,
  "P0": 0,
  "Gamma": 1,
  "weightavg": True,
  "perfectagent": False,
  "evalN": 1000}
    
Pv["TD3_kwargs"]={
    "discount": 0.99,
    "tau": 0.005,
    "policy_noise": 0.05,
    "noise_clip": 0.5,
    "policy_freq": 2,
    "lr": 3e-4} #Learning rate

Pv["model_kwargs"]={
    "num_networks": 7,
    "num_elites": 5,
    "model_lr": 0.001}

# Agent hyper-parameters
v["market_kwargs"] = Pv["market_kwargs"]
v["market_kwargs"]["Contract"] = np.ones(T)/T #This doesn't do anything, it's overwritten during training but needs to be there for initialization
v["market_kwargs"]["tol"] = 1
v["market_kwargs"]["minut"] = -5 #This helps keep the gradients somewhat reasonable
v["market_kwargs"]["start_timesteps"] = 500*(T-1) # 500 episodes
v["train_kwargs"]["max_timesteps"] = 1000000
v["train_kwargs"]["max_episodes"] = 4000
v["train_kwargs"]["eval_freq"] = 100*(T-1) # Evaluate every 100 episodes
v["train_kwargs"]["verbose"] = False
learnpri = MBPOP(train_kwargs=Pv["train_kwargs"], model_kwargs=Pv["model_kwargs"], TD3_kwargs=Pv["TD3_kwargs"],market_kwargs=Pv["market_kwargs"],agent_v = v)

---------------------------------------
Policy: TD3, Env: Agent, Seed: 0
---------------------------------------


In [10]:
learnpri.init_models_and_buffer()

---------------------------------------
Policy: TD3, Env: Principal, Seed: 0
---------------------------------------


In [ ]:
learnpri.train()

     Beginning to train agent with new contract
---------------------------------------
Agent Evaluation over 1 episodes: -4.236
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -4.236
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -4.230
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -4.237
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -4.231
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -4.254
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -2.029
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.085
------------------------

In [ ]:
learnpri.train()

Streaming output truncated to the last 5000 lines.
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.216
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.216
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.190
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.207
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.235
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.223
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.201
---------------------------------------
---------------------------------------
Agent Evaluation over 

In [ ]:
learnpri.train()

Streaming output truncated to the last 5000 lines.
---------------------------------------
Agent Evaluation over 1 episodes: -1.142
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.142
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.143
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.132
---------------------------------------
---------------------------------------
Agent Evaluation over 1 episodes: -1.145
---------------------------------------
     Agent converged successfully
---------------------------------------
Agent Evaluation over 1 episodes: -1.145
---------------------------------------
Principal Episode Num: 10 Episode T: 1 Reward: -0.931
     Beginning to train agent with new contract
---------------------------------------
Agent Evaluation over 1 episodes: -1.043
------